In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 421.8MB 39kB/s 
     |████████████████████████████████| 3.8MB 55.2MB/s 
     |████████████████████████████████| 450kB 50.0MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=7ba49783e60b57388fa276175d74f69b3b4012587f87fc64bf86778f61f3aaed
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
ERROR: tensorflow 2.1.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.3.3 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    

In [0]:
#from tensorflow.python.keras.models import Sequential
#from tensorflow.python.keras.layers import Dense,Dropout,LeakyReLU,PReLU,ELU

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_
n_test=math.sqrt(n)X_val=X_val.reshape((int(mval),int(n_test),int(n_test)))

Using TensorFlow backend.


In [0]:
!pip install keras-tuner
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras

X_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/miteshsir/test.csv")
X_val=pd.read_csv("/content/drive/My Drive/Colab Notebooks/miteshsir/val.csv")
X_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/miteshsir/train.csv')

y_train=X_train['label']
y_val=X_val['label']


X_test=X_test.drop(['id'],axis=1)
X_train=X_train.drop(['id','label'],axis=1)
X_val=X_val.drop(['id','label'],axis=1)


X_train=X_train.to_numpy(dtype=int)
X_test=X_test.to_numpy(dtype=int)
X_val=X_val.to_numpy(dtype=int)



mtest=X_test.shape[0]
n=X_test.shape[1]
mtrain=X_train.shape[0]
mval=X_val.shape[0]
import math
n_test=math.sqrt(n)


'''from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)
X_val=sc.fit_transform(X_val)'''

X_train=X_train/255
X_test=X_test/255
X_val=X_val/255

X_train=X_train.reshape((int(mtrain),1,int(n_test),int(n_test)))
X_test=X_test.reshape((int(mtest),1,int(n_test),int(n_test)))
X_val=X_val.reshape((int(mval),1,int(n_test),int(n_test)))
#X_train=np.vstack((X_train,X_val))


from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,MaxPool2D
from keras.layers import PReLU,LeakyReLU,ELU
from keras import backend as K

K.common.image_dim_ordering()
K.common.set_image_dim_ordering(dim_ordering='th')

def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(filters=64,
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              input_shape=(1,int(n_test),int(n_test)),
                                              kernel_size=(3,3),
                                              data_format='channels_first'
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first',
                                              filters=128
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first'
                                              ),
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first'
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(units=1024,input_dim=256,activation='relu',kernel_initializer='he_uniform'),
                          keras.layers.Dense(units=1024,input_dim=1024,activation='relu',kernel_initializer='he_uniform'),
                          

                          keras.layers.Dense(units=10,input_dim=1024,activation='softmax',kernel_initializer='glorot_uniform')

                                                  
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learningrate',values=[1e-2,1e-3,1e-1,1e-4])),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  model.summary()
  return model

from kerastuner import RandomSearch
import kerastuner.engine.hyperparameters as Hyperparametrs


tuner_search=RandomSearch(build_model,
                          objective='accuracy',
                          max_trials=5,
                          directory="save_dir"
                          )

tuner_search.search(X_train,y_train,epochs=3)

model=tuner_search.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=10,initial_epoch=3,batch_size=10)

y_pred1=model.predict_classes(X_val)
y_test=model.predict_classes(X_test)
from sklearn.metrics import accuracy_score
score1=accuracy_score(y_pred1,y_val)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred1,y_val)








ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-34ee3992bbfa>", line 2, in <module>
    from kerastuner import RandomSearch
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/__init__.py", line 19, in <module>
    from kerastuner import applications
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/applications/__init__.py", line 17, in <module>
    from .resnet import HyperResNet
  File "/usr/local/lib/python3.6/dist-packages/kerastuner/applications/resnet.py", line 19, in <module>
    from tensorflow.keras import layers
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/pyt

ImportError: ignored

In [0]:
!pip install keras-tuner
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras

X_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/miteshsir/test.csv")
X_val=pd.read_csv("/content/drive/My Drive/Colab Notebooks/miteshsir/val.csv")
X_train=pd.read_csv('/content/drive/My Drive/Colab Notebooks/miteshsir/train.csv')

y_train=X_train['label']
y_val=X_val['label']


X_test=X_test.drop(['id'],axis=1)
X_train=X_train.drop(['id','label'],axis=1)
X_val=X_val.drop(['id','label'],axis=1)


X_train=X_train.to_numpy(dtype=int)
X_test=X_test.to_numpy(dtype=int)
X_val=X_val.to_numpy(dtype=int)



mtest=X_test.shape[0]
n=X_test.shape[1]
mtrain=X_train.shape[0]
mval=X_val.shape[0]
import math
n_test=math.sqrt(n)


from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)
X_val=sc.fit_transform(X_val)

X_train=X_train.reshape((int(mtrain),1,int(n_test),int(n_test)))
X_test=X_test.reshape((int(mtest),1,int(n_test),int(n_test)))
X_val=X_val.reshape((int(mval),1,int(n_test),int(n_test)))


from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,MaxPool2D
from keras.layers import PReLU,LeakyReLU,ELU
from keras import backend as K

K.common.image_dim_ordering()
K.common.set_image_dim_ordering(dim_ordering='th')

def build_model(hp):
  model=keras.Sequential([
                          keras.layers.Conv2D(filters=64,
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              input_shape=(1,int(n_test),int(n_test)),
                                              kernel_size=(3,3),
                                              data_format='channels_first'
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first',
                                              filters=128
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first'
                                              ),
                          keras.layers.Conv2D(filters=256,
                                              kernel_size=(3,3),
                                              kernel_initializer='he_uniform',
                                              strides=(1,1),
                                              activation='relu',
                                              data_format='channels_first'
                                              ),
                          keras.layers.MaxPool2D(pool_size=(2,2),padding='same'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(units=1024,input_dim=256,activation='relu',kernel_initializer='he_uniform'),
                          keras.layers.Dense(units=1024,input_dim=1024,activation='relu',kernel_initializer='he_uniform'),
                          

                          keras.layers.Dense(units=10,input_dim=1024,activation='softmax',kernel_initializer='glorot_uniform')

                                                  
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learningrate',values=[1e-2,1e-3,1e-1,1e-4])),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])
  model.summary()
  return model

from kerastuner import RandomSearch
import kerastuner.engine.hyperparameters as Hyperparametrs


tuner_search=RandomSearch(build_model,
                          objective='accuracy',
                          max_trials=5
                          )

tuner_search.search(X_train,y_train,epochs=3)

model=tuner_search.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=10,initial_epoch=3,batch_size=10)

y_pred1=model.predict_classes(X_val)
y_test=model.predict_classes(X_test)
from sklearn.metrics import accuracy_score
score1=accuracy_score(y_pred1,y_val)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred1,y_val)

     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 3.8MB 52.7MB/s 
     |████████████████████████████████| 450kB 62.0MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15354 sha256=86cee31dc5686757ce1a250017bdb648119896400828633e747e6f6beb0e0043
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
ERROR: tensorflow 2.1.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.3.3 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    U

FileNotFoundError: ignored